In [ ]:
#adjusting code learnt on udemy course

In [16]:
#load tape data 
import pandas as pd

tape = pd.read_csv('tape.csv')

In [17]:
tape.head()

,Time,Price,Quantity,Date,Normalised_Price,Standardised_Price,TimeDelta
0,11.067,269,1,2025-01-02,0.578947,0.855046,0 days 11:04:01.200000
1,11.222,267,2,2025-01-02,0.572368,0.830077,0 days 11:13:19.200000
2,12.338,270,2,2025-01-02,0.582237,0.867531,0 days 12:20:16.800000
3,13.733,267,3,2025-01-02,0.572368,0.830077,0 days 13:43:58.800000
4,18.321,265,2,2025-01-02,0.565789,0.805109,0 days 18:19:15.600000


In [18]:
tape['Date'] = pd.to_datetime(tape['Date'])

data = tape.groupby('Date').agg(
    Open=('Price', 'first'),
    High=('Price', 'max'),
    Low=('Price', 'min'),
    Close=('Price', 'last'),
    Adj_Close=('Normalised_Price', 'last'), 
    Volume=('Quantity', 'sum')
).reset_index()

In [19]:
data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2025-01-02,269,303,243,288,0.641447,47972
1,2025-01-03,283,317,270,291,0.651316,51991
2,2025-01-06,278,346,276,327,0.769737,48045
3,2025-01-07,336,388,317,369,0.907895,44332
4,2025-01-08,373,374,303,311,0.717105,44856


In [20]:
#backtesting packages
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [21]:
#sma function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 30
    n2 = 100

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
#import yfinance as yf

# Get BTC-USD data from yfinance
#data = yf.download('BTC-USD')


In [22]:
# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


C:\Users\s_sta\AppData\Local\Temp\ipykernel_29448\968553843.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, SmaCross)


In [23]:
data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2025-01-02,269,303,243,288,0.641447,47972
1,2025-01-03,283,317,270,291,0.651316,51991
2,2025-01-06,278,346,276,327,0.769737,48045
3,2025-01-07,336,388,317,369,0.907895,44332
4,2025-01-08,373,374,303,311,0.717105,44856


In [24]:
#print(stats['_trades'].to_string())
stats

Start                                     0.0
End                                     124.0
Duration                                124.0
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              -62.847222
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [25]:
bt.plot()

Row(id='1973', ...)

In [26]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 60, 5), 'n2': range(10, 90, 5)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

  0%|          | 0/13 [00:00<?, ?it/s]

Best result:  42.230000000000004
Parameters for best result:  SmaCross(n1=40,n2=50)


In [27]:
res

Start                                     0.0
End                                     124.0
Duration                                124.0
Exposure Time [%]                        16.8
Equity Final [$]                      14223.0
Equity Peak [$]                       14223.0
Return [%]                              42.23
Buy & Hold Return [%]              -62.847222
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.363662
Avg. Drawdown [%]                   -4.871815
Max. Drawdown Duration                    7.0
Avg. Drawdown Duration                    3.0
# Trades                                  2.0
Win Rate [%]                            100.0
Best Trade [%]                      19.587629
Worst Trade [%]                     18.965517
Avg. Trade [%]                    

In [28]:
res['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,103,104,108,97.0,116,1957.0,0.195876,104,108,4
1,-103,108,124,116.0,94,2266.0,0.189655,108,124,16


In [29]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
0,10000.0,0.000000,NaN
1,10000.0,0.000000,NaN
2,10000.0,0.000000,NaN
3,10000.0,0.000000,NaN
4,10000.0,0.000000,NaN
...,...,...,...
120,13193.0,0.051818,NaN
121,12472.0,0.103637,NaN
122,12472.0,0.103637,NaN
123,12575.0,0.096234,NaN
